In [159]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException, NoSuchWindowException, StaleElementReferenceException
from bs4 import BeautifulSoup
import time 
import random
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://www.daraz.pk/catalog/?q=Redmi+Note+8&_keyori=ss&from=input&spm=a2a0e.searchlist.search.go.663b1f96cRxGog')

prodDictionary = dict()
prodDictionary['Product Name'] = []
prodDictionary['Images Links'] = []
prodDictionary['Daraz Links'] = []
prodDictionary['Pricing'] = []
prodDictionary['Seller Rating'] = []
prodDictionary['Reviews Score'] = []
prodDictionary['Total Reviews'] = []
prodDictionary['Storage Capacity'] = []


In [160]:
def gettingAllCards():
    return driver.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

In [161]:
all_links = []
def GetProductDetails():
    #once we get the cards we are going to extract vital info such as 
    #links <--> Product Name <--> Image Link <--> Daraz Links <--> Pricing 
    all_page_cards = gettingAllCards()
    #the testpage is the card 
    #we can get 2 links from here 1 img link and the other one is the link to the product
    #Getting the title and the product price
    to_exclude = ['charger', 'case', 'cover']
    limit = 10000
    for card in all_page_cards:
        title = card.find_element(By.CLASS_NAME,'title--wFj93').text
        title_tokens = nltk.word_tokenize(title.lower())
        for token in title_tokens:
            if token in to_exclude:
                return 
        prodDictionary['Product Name'].append(card.find_element(By.CLASS_NAME, 'title--wFj93').text)
        prodDictionary['Pricing'].append(card.find_element(By.CLASS_NAME, 'currency--GVKjl').text)
        daraz_link = card.find_element(By.ID, 'id-a-link') #daraz link to product
        prodDictionary['Daraz Links'].append(daraz_link.get_attribute('href'))
        img_link = daraz_link.find_element(By.ID, 'id-img')
        prodDictionary['Images Links'].append(img_link.get_attribute('src'))
    

In [162]:
#getting the next page links:
def nextPage(pageNumber):
    nextpagelink = driver.find_element(By.CLASS_NAME, 'pager--mb0Ws')
    specific_page_link = nextpagelink.find_elements(By.TAG_NAME, 'li')
    #when page number is 0 then its on page #2
    links_specific_pages = specific_page_link[pageNumber].find_elements(By.TAG_NAME,'a')
    return links_specific_pages[0]

In [163]:
for page_num in range(1,6):
    GetProductDetails()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    next_page_link = nextPage(page_num).click()

In [164]:
# getting the reviews and links details
def scrollSlowly(driver):
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(500, 800);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(800, 1000);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(1000, 1300);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(1300, 1600);")
    time.sleep(1)
    
def getReviews():
    allLinks = prodDictionary['Daraz Links']
    for link in allLinks:
        driver.get(link)
        time.sleep(2)
        infoBox = driver.find_element(By.ID, 'block-D5ZgSG8FxRO')
        sellerBox = infoBox.find_element(By.CLASS_NAME, 'pdp-seller-info-pc')
        seller_ratings = sellerBox.find_element(By.CLASS_NAME, 'info-content')
        prodDictionary['Seller Rating'].append(seller_ratings.text.split('\n')[1])
        try:
            storage = driver.find_element(By.XPATH, '//*[@id="module_sku-select"]/div/div[2]/div/div/div[1]/span').text
        except NoSuchElementException:
            storage = 'Not Specified'
            
        prodDictionary['Storage Capacity'].append(storage)
        scrollSlowly(driver)
        
        try:
            reviewsBox = driver.find_element(By.CLASS_NAME, 'review-info-left')
            review_score = reviewsBox.find_element(By.CLASS_NAME, 'score').text
        except NoSuchElementException:
            review_score = "No Reviews"
        
        prodDictionary['Reviews Score'].append(review_score)
        
        try:
            numOfrevs = reviewsBox.find_element(By.CLASS_NAME, 'rate-num').text.split(' ')[0]
        except (NoSuchElementException, StaleElementReferenceException,UnboundLocalError ):
            numOfrevs = '0'
        prodDictionary['Total Reviews'].append(numOfrevs)

In [165]:
#scraped total 185
getReviews()

In [166]:
prodDictionary

{'Product Name': ['Xiaomi Redmi Note 12 RAM 6 GB ROM 128 Front Camera 13 MP Back Camera 50 MP + 8 MP + 2 MP Battery Type Li-Po 5000 mAh, non-removable',
  'Xiaomi Redmi Note 12 RAM 6 GB ROM 128 Front Camera 13 MP Back Camera 50 MP + 8 MP + 2 MP Battery Type Li-Po 5000 mAh, non-removable',
  'Redmi Note 12 8 GB RAM + 128GB ROM PTA Approved Mobile Phone with 1 Year Brand Warranty',
  'Redmi Note 12 8GB Ram 128GB Rom, Display Size 6.67 Inches, Battery Capacity 5000 mAh , Fast charging 33W',
  'Redmi Note 12 || 8GB Ram 128GB Rom || 6.67 Inches AMOLED Display || 5000 mAh - Fast charging 33W wired ||',
  'Redmi Note 12s - 6.43" Inch Display - 8GB RAM - 256GB ROM',
  'Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB Front Camera 16 MP Back Camera 108 MP + 08 MP + 02 MP Battery Type Li-Po 5000 mAh, non-removable',
  'Redmi Note 12 Pro - 6.67" Inch Display - 8GB RAM - 256GB ROM',
  'Redmi Note 12 - 6.67" Inch Display - 8GB RAM - 128GB / 256GB ROM',
  'Xiaomi Redmi Note 12 RAM 8 GB ROM 128 Front Cam

In [168]:
dfProducts = pd.DataFrame(prodDictionary)
dfProducts['ID'] = [i for i in range((dfProducts.shape[0]))]
dfProducts

,Product Name,Images Links,Daraz Links,Pricing,Seller Rating,Reviews Score,Total Reviews,Storage Capacity,ID
0,Xiaomi Redmi Note 12 RAM 6 GB ROM 128 Front Ca...,https://static-01.daraz.pk/p/1b03e14fe20e15ac0...,https://www.daraz.pk/products/12-6-gb-128-13-m...,"Rs. 59,980",95%,No Reviews,0,128GB,0
1,Xiaomi Redmi Note 12 RAM 6 GB ROM 128 Front Ca...,https://static-01.daraz.pk/p/cad65545dfc2ba35a...,https://www.daraz.pk/products/12-6-gb-128-13-m...,"Rs. 59,880",95%,5.0,1,128GB,1
2,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,https://static-01.daraz.pk/p/67d0213e9891004b6...,https://www.daraz.pk/products/redmi-note-12-8-...,"Rs. 64,999",92%,4.7,396,128GB,2
3,"Redmi Note 12 8GB Ram 128GB Rom, Display Size ...","data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",https://www.daraz.pk/products/12-8gb-128gb-667...,"Rs. 49,999",94%,4.7,11,128GB,3
4,Redmi Note 12 || 8GB Ram 128GB Rom || 6.67 Inc...,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",https://www.daraz.pk/products/12-8gb-1-000-gb-...,"Rs. 51,399",90%,4.7,23,128GB,4
...,...,...,...,...,...,...,...,...,...
85,Redmi Note 12 - 6GB Ran 128GB Rom - 5000mAh Ba...,https://static-01.daraz.pk/p/7e6f6c4c134d8ba87...,https://www.daraz.pk/products/12-6gb-ran-128gb...,"Rs. 64,999",91%,3.0,2,128GB,85
86,Xiaomi Redmi Note 12 RAM 6 GB ROM 128 Front Ca...,https://static-01.daraz.pk/p/cad65545dfc2ba35a...,https://www.daraz.pk/products/12-6-gb-128-13-m...,"Rs. 51,890",95%,No Reviews,0,128GB,86
87,Redmi Note 12 - 8 GB Ram - 128GB Rom PTA Approved,https://static-01.daraz.pk/p/9fd5f4a1c5e576b45...,https://www.daraz.pk/products/12-8-gb-128gb-pt...,"Rs. 53,000",95%,No Reviews,0,Not Specified,87
88,"Redmi Note 12 8GB Ram 128GB Rom, Display Size ...","data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",https://www.daraz.pk/products/12-8gb-128gb-667...,"Rs. 53,999",Not enough data,No Reviews,0,128GB,88


In [172]:
filtered_links = dfProducts['Daraz Links']
filtered_links

0     https://www.daraz.pk/products/12-6-gb-128-13-m...
1     https://www.daraz.pk/products/12-6-gb-128-13-m...
2     https://www.daraz.pk/products/redmi-note-12-8-...
3     https://www.daraz.pk/products/12-8gb-128gb-667...
4     https://www.daraz.pk/products/12-8gb-1-000-gb-...
                            ...                        
85    https://www.daraz.pk/products/12-6gb-ran-128gb...
86    https://www.daraz.pk/products/12-6-gb-128-13-m...
87    https://www.daraz.pk/products/12-8-gb-128gb-pt...
88    https://www.daraz.pk/products/12-8gb-128gb-667...
89    https://www.daraz.pk/products/12-8-gb-256-gb-1...
Name: Daraz Links, Length: 90, dtype: object

In [174]:
page_ctr = 2
reviews = dict()
reviews['ID'] = []
reviews['Reviewer Name'] = []
reviews['Review'] = []

for link, id_ in zip(filtered_links, dfProducts['ID']):
    driver.get(link)
    scrollSlowly(driver)
    try:
        reviewBox = driver.find_element(By.CLASS_NAME, "review-content")
    except NoSuchElementException:
        print('No Reviews')
        continue
    #getting all reviews in page-1
    all_page_revs = reviewBox.find_elements(By.CLASS_NAME, 'review-item')
    #getting the name of reviewer
    for rev in all_page_revs:
        check_name=rev.find_element(By.CLASS_NAME, 'review-content-fl')
        name = check_name.find_element(By.CLASS_NAME, 'user').text
        reviews['Reviewer Name'].append(name)
        #getting the actual review of the reviewer
        check_review =rev.find_element(By.CLASS_NAME, 'review-content-sl').text
        reviews['Review'].append(check_review)
        reviews['ID'].append(id_)
    #getting to other page
    revBox = driver.find_element(By.CLASS_NAME, 'review-pagination')
    next_page = revBox.find_element(By.TAG_NAME, 'ul')
    all_pages = next_page.find_elements(By.TAG_NAME, 'li')
    #the click() does not come on the second last element: starts at (1-7){
    # {1 Start, all_pages-2 } the pages will work till this
    if page_ctr > len(all_pages)-2 or len(all_pages) == 2:
        continue
    else:
        all_pages[page_ctr].find_element(By.TAG_NAME, 'a').click()
        page_ctr += 1

No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews
No Reviews


In [177]:
reviewsDF = pd.DataFrame(reviews)
reviewsDF

,ID,Reviewer Name,Review
0,1,M.Sharjeel R.,I got the parcel packed securely and the origi...
1,2,Gohar I.,this is the initial review. the phone is the s...
2,2,Yasir Q.,Daraz is very helpful😎😎I was not have enough m...
3,2,3***4,"Camera satisfactory nh ha, Baki mobile ok ha"
4,3,Hamza R.,It was just awesome thanks daraz thank you so ...
...,...,...,...
94,81,Aslam K.,The phone was received well on time without an...
95,81,Waleed N.,Product was well packed and of good quality. O...
96,84,3***3,excellent 👍 service saller cooperative product...
97,85,Irfan A.,بہت ہی گھٹیا بھائی صاحب سے موبائل خریدا جن کو ...


In [190]:
total_csv = pd.merge(dfProducts, reviewsDF, on='ID', how='inner')

In [191]:
total_csv.to_csv('main.csv', index = False)

In [192]:
total_csv


,Product Name,Images Links,Daraz Links,Pricing,Seller Rating,Reviews Score,Total Reviews,Storage Capacity,ID,Reviewer Name,Review
0,Xiaomi Redmi Note 12 RAM 6 GB ROM 128 Front Ca...,https://static-01.daraz.pk/p/cad65545dfc2ba35a...,https://www.daraz.pk/products/12-6-gb-128-13-m...,"Rs. 59,880",95%,5.0,1,128GB,1,M.Sharjeel R.,I got the parcel packed securely and the origi...
1,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,https://static-01.daraz.pk/p/67d0213e9891004b6...,https://www.daraz.pk/products/redmi-note-12-8-...,"Rs. 64,999",92%,4.7,396,128GB,2,Gohar I.,this is the initial review. the phone is the s...
2,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,https://static-01.daraz.pk/p/67d0213e9891004b6...,https://www.daraz.pk/products/redmi-note-12-8-...,"Rs. 64,999",92%,4.7,396,128GB,2,Yasir Q.,Daraz is very helpful😎😎I was not have enough m...
3,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,https://static-01.daraz.pk/p/67d0213e9891004b6...,https://www.daraz.pk/products/redmi-note-12-8-...,"Rs. 64,999",92%,4.7,396,128GB,2,3***4,"Camera satisfactory nh ha, Baki mobile ok ha"
4,"Redmi Note 12 8GB Ram 128GB Rom, Display Size ...","data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",https://www.daraz.pk/products/12-8gb-128gb-667...,"Rs. 49,999",94%,4.7,11,128GB,3,Hamza R.,It was just awesome thanks daraz thank you so ...
...,...,...,...,...,...,...,...,...,...,...,...
94,Xiaomi Redmi Note 12 RAM 8 GB ROM 128 Front Ca...,https://static-01.daraz.pk/p/389f7befa963be9ae...,https://www.daraz.pk/products/12-8-gb-128-13-m...,"Rs. 52,500",93%,4.8,5,128GB,81,Aslam K.,The phone was received well on time without an...
95,Xiaomi Redmi Note 12 RAM 8 GB ROM 128 Front Ca...,https://static-01.daraz.pk/p/389f7befa963be9ae...,https://www.daraz.pk/products/12-8-gb-128-13-m...,"Rs. 52,500",93%,4.8,5,128GB,81,Waleed N.,Product was well packed and of good quality. O...
96,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,https://static-01.daraz.pk/p/93a4dee5f0780599c...,https://www.daraz.pk/products/12-8-gb-256-gb-1...,"Rs. 94,990",95%,5.0,1,256GB,84,3***3,excellent 👍 service saller cooperative product...
97,Redmi Note 12 - 6GB Ran 128GB Rom - 5000mAh Ba...,https://static-01.daraz.pk/p/7e6f6c4c134d8ba87...,https://www.daraz.pk/products/12-6gb-ran-128gb...,"Rs. 64,999",91%,3.0,2,128GB,85,Irfan A.,بہت ہی گھٹیا بھائی صاحب سے موبائل خریدا جن کو ...


In [193]:
def convert_pricing(price_str):
    try:
        # Extract the numeric part and convert to integer
        return int(price_str.split()[-1].replace(',', ''))
    except ValueError:
        # Handle cases where conversion fails (e.g., non-numeric values)
        return None

# Function to convert 'Reviews Score' column to integer
def convert_reviews_score(score_str):
    try:
        # Remove '%' and convert to integer
        return int(score_str.rstrip('%'))
    except ValueError:
        return None

# Function to convert 'Storage Capacity' column to integer
def convert_storage_capacity(capacity_str):
    try:
        # Extract the numeric part and convert to integer
        return int(capacity_str.split('GB')[0])
    except ValueError:
        return None
def convert_reviews_score(score_str):
    try:
        # Remove '%' and convert to float
        return float(score_str)
    except ValueError:
        return None

# Apply the conversion function to the 'Reviews Score' column
total_csv['Reviews Score'] = total_csv['Reviews Score'].apply(convert_reviews_score)
# Apply the conversion functions to the respective columns
total_csv['Pricing'] = total_csv['Pricing'].apply(convert_pricing)
total_csv['Seller Rating'] = total_csv['Reviews Score'].apply(convert_reviews_score)
total_csv['Storage Capacity'] = total_csv['Storage Capacity'].apply(convert_storage_capacity)

In [197]:
total_csv.to_csv('final.csv', index=False)